In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import time
import datetime
import re
import urllib
from bs4 import BeautifulSoup
from random import randint
import pyLDAvis.gensim
from pprint import pprint

In [3]:
initial_date_str = '1 Jan 2018'
initial_date = datetime.datetime.strptime(initial_date_str, '%d %b %Y')
final_date_str = '13 Jun 2018'

final_date = datetime.datetime.strptime(final_date_str, '%d %b %Y')
current_date = initial_date

In [4]:
tweets_list = []
while current_date < final_date :        
    aux_list = open('Tweets/' + current_date.strftime("%Y-%m-%d") + '_trump.txt',encoding='UTF-8').read().split('\n')
    aux_list = [current_date.strftime("%Y-%m-%d") + ', ' + row for row in aux_list]
    tweets_list.extend(aux_list)
    current_date = current_date + datetime.timedelta(days=1)

In [5]:
tweets_list[6500]

'2018-04-15, JTStevison, https://twitter.com/JTStevison/status/985656472687259649, 1, 11, 4, Perfect. <img class="Emoji Emoji--forText" src="https://abs.twimg.com/emoji/v2/72x72/1f602.png" draggable="false" alt="😂" title="Face with tears of joy" aria-label="Emoji: Face with tears of joy"><img class="Emoji Emoji--forText" src="https://abs.twimg.com/emoji/v2/72x72/1f602.png" draggable="false" alt="😂" title="Face with tears of joy" aria-label="Emoji: Face with tears of joy"><a href="/hashtag/DonaldTrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b><strong>DonaldTrump</strong></b></a> <a href="/hashtag/Syria?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Syria</b></a> <a href="/hashtag/SyrianStrikes?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>SyrianStrikes</b></a> <a href="/hashtag/Trump?src=hash" data-qu

In [6]:
def add_space_between_a_tags(html):
    pos_1 = html.find('<a')
    while pos_1 != -1:
        html = html[:pos_1] + ' ' + html[pos_1:]
        pos_1 = html.find('<a',pos_1+2)
        
    pos_1 = html.find('</a>')
    while pos_1 != -1:
        html = html[:pos_1+4] + ' ' + html[pos_1+4:]
        #if len(html) == html.find('</a>',pos_1 + 4)
        pos_1 = html.find('</a>',pos_1+4)
        
    return html

In [7]:
print(add_space_between_a_tags(tweets_list[9870]))

2018-06-09, reggaerapstarCP, https://twitter.com/reggaerapstarCP/status/1005393266530021379, 0, 1, 3, Remember this?This This  <a href="/hashtag/Original?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Original</b></a>   <a href="/hashtag/hand?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>hand</b></a>   <a href="/hashtag/painting?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>painting</b></a>   <a href="/hashtag/handdesigned?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>handdesigned</b></a>   <a href="/hashtag/usa?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>usa</b></a>   <a href="/hashtag/presidenttrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link j

In [8]:
print(tweets_list[9870])

2018-06-09, reggaerapstarCP, https://twitter.com/reggaerapstarCP/status/1005393266530021379, 0, 1, 3, Remember this?This This <a href="/hashtag/Original?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Original</b></a> <a href="/hashtag/hand?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>hand</b></a> <a href="/hashtag/painting?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>painting</b></a> <a href="/hashtag/handdesigned?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>handdesigned</b></a> <a href="/hashtag/usa?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>usa</b></a> <a href="/hashtag/presidenttrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir=

In [9]:
def get_hashtags(tweet):
    hashtags = []
    pos = tweet.find('#')
    while pos != -1:
        pos_aux = tweet.find(" ",pos)
        if pos_aux == -1:
            break
        hashtags.append(tweet[pos:pos_aux])
        tweet = tweet[:pos] + ' ' + tweet[pos_aux:]
        pos = tweet.find('#',pos + 1)
    return hashtags

In [10]:
print(get_hashtags("Body Language Analysis No.4303: Donald Trump, G7 Summit, Women's Empowerment, and Rage -  #Nonverbal &  #EmotionalIntelligence •   •  #DonaldTrump #AngelaMerkel #Anger #BehaviorAnalysis #BodyLanguage #BodyLanguageExpert #Analysis #EQ #G7 #G7Summit"))

['#Nonverbal', '#EmotionalIntelligence', '#DonaldTrump', '#AngelaMerkel', '#Anger', '#BehaviorAnalysis', '#BodyLanguage', '#BodyLanguageExpert', '#Analysis', '#EQ', '#G7']


In [11]:
def get_mentions(tweet):
    mentions = []
    pos = tweet.find('@')
    while pos != -1:
        pos_aux = tweet.find(" ",pos)
        if pos_aux == -1:
            break
        mentions.append(tweet[pos:pos_aux])
        tweet = tweet[:pos] + ' ' + tweet[pos_aux:]
        pos = tweet.find('@',pos + 1)
    return mentions

In [12]:
print(get_mentions("May God Bless you and Bless God for  #DonaldTrump @lindathomas102 @louisa_ip @maga_417 @MarcGriff89 @MarjorieLoy1@Maximus_Paulus @MC_Mad_Man @mgw81180 @MrSteveRangel @MyMountainTop @Namashary @netteKnows @NotUnderdog @NYKchi88 @nykon187@ofarther"))

['@lindathomas102', '@louisa_ip', '@maga_417', '@MarcGriff89', '@MarjorieLoy1@Maximus_Paulus', '@MC_Mad_Man', '@mgw81180', '@MrSteveRangel', '@MyMountainTop', '@Namashary', '@netteKnows', '@NotUnderdog', '@NYKchi88']


In [13]:
dates = []
user = []
status = []
comments = []
likes = []
retweets = []
tweet = []
tweet_no_html = []
hashtags = []
mentions = []
topic_words_per_document = []
for tweet_status in tweets_list:
    pos = tweet_status.find(',')
    dates.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    user.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    status.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    comments.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    likes.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    retweets.append(tweet_status[:pos])
        
    tweet_status = tweet_status[pos + 1:].strip()    
    soup = BeautifulSoup(add_space_between_a_tags(tweet_status),'lxml')
    
    tweet_status_no_html = soup.text
    
    tweet.append(tweet_status_no_html)
    
    pos_2 = tweet_status_no_html.find('pic.twitter.com/')
    tweet_status_no_html = tweet_status_no_html[:pos_2].strip()
    pos_3 = tweet_status_no_html.find('http')
    while pos_3 != -1:        
        tweet_status_no_html = tweet_status_no_html[:pos_3 - 1] + ' ' + tweet_status_no_html[pos_3:]
        pos_3 = tweet_status_no_html.find('http',pos_3 + 1)
    tweet_status_no_html = re.sub(r"http\S+", "", tweet_status_no_html)
    tweet_status_no_html = tweet_status_no_html.strip().replace("\xa0","")
    tweet_no_html.append(tweet_status_no_html)
    
    hashtags.append(get_hashtags(tweet_status_no_html))
    mentions.append(get_mentions(tweet_status_no_html))

In [14]:
len(dates)

10244

In [15]:
x = randint(0, len(dates))
print(x)
print(dates[x],user[x],status[x],comments[x],likes[x],retweets[x],tweet[x],tweet_no_html[x],hashtags[x],mentions[x],end="\n")

8628
2018-05-22 ThePerezHilton https://twitter.com/ThePerezHilton/status/999048403655913473 4 17 1 Free yourself, Melanie!!  https://goo.gl/3NfrWJ  #MelaniaTrump #DonaldTrump #MAGA  Free yourself, Melanie!!   #MelaniaTrump #DonaldTrump #MAGA ['#MelaniaTrump', '#DonaldTrump'] []


In [16]:
index = 0
for i in tweet_no_html:
    print(index,':',i)
    index = index + 1

0 : Dear  #DonaldTrump
1 : #America  demands  #TrumpShowUsYourPenis #DonaldTrump #MAGA
2 : Enough.  #DonaldTrump
3 : #DonaldTrump  stop blaming PAK... and PAK Should  #StopAmericanSupplies
4 : Pakistan will Not Do More !!  #NoMoreTrump #DonaldTrump #StopAmericanSupplies
5 : #DonaldTrump #DonaldTrumpsPak  Short term memory loss syndrome OR Bipolar disorder?
6 : Pakistan is not Fighting for Money : DG ISPR #NoMoreTrump #StopAmericanSupplies #DonaldTrump
7 : Dear  #DonaldTrump  you know fairly after March 2018.......you will never be able to threatened State of Pakistan ever again.......So now  #NoMoreTrump
8 : So, the President of America doesn't know that the exeggerated figure of $33 billion in Coalition Support Fund "IS NOT AID".  #DonaldTrump
9 : Only problem is .... #DonaldTrump  can't be disappeare
10 : #DonaldTrump  is trending top in  #Pakistan  right now and not for any good reason
11 : Becoz we are busy in lifting "Mukka" from Mukka chawk..all 33 billion dollars are used to van

959 : #DonaldTrump  partly denies, also defends vulgar immigrant comments  …
960 : No Shit,  #donaldtrump  Is Racist
961 : DeAr BLACK Africans!!! You deserve what ever stupid people like the  #crackhead #DonaldTRUMP  refers to you...
962 : 'mum why did  #DonaldTrump  say he wants people from  #Norway  to go to America?''because he only wants white people to go there.''Oh. why?''Because he is a racist' @realDonaldTrump #shitholecountry
963 : as does Paul Ryan folks...  @SpeakerRyan #donaldtrump #trump #trumptrain #maga #WhiteSupremacy  Trump The Pig is fried
964 : UN rights office decries  #DonaldTrump 's reported remarks as 'racist'  #President #UnitedNations ...  …
965 : Hmmm okay  #donaldtrump  #celebrity #celebrities #trending #trendingtopic #whitehouse …
966 : Trump lawyer arranged pre-election hush  #money  for porn star: report   #global #donaldtrump
967 : 
968 : Donald bumbohole dump  #DonaldTrump #shithole
969 : #Nicaragua : For Arrogant  #DonaldTrump , probably just another  #

1959 : RETWEET BEFORE ITS TAKEN DOWN *** HELICOPTER JUST CRASHED IN THE BLACK FOREST HERE IS THE LINK***   #Q #qanon #qanon8chan #FolllowTheWhiteRabbit #8chan #4cha #MegaAnon #mega #DonaldTrump #RETWEEET
1960 : I know my opinion will mean nothing but I thought the  @piersmorgan  interview with  #DonaldTrump  was really well handled and just the right amount of serious conversation and some humour. Well done Piers, great work Sir
1961 : #DonaldTrump  has officially become all that he admires, a hostile corrupt dictator.
1962 : #Republicans  ordered an investigation into the  #FBI  and the entire DOJ because they won't do  #DonaldTrump 's bidding. This is no longer a democratic republic. We are now a third-world nation. EVERY SINGLE REPUBLICAN IS COMPLICIT.  #ProtectMueller #ProtectRosenstein #Treason #FBR
1963 : BRAEKING: FOX NEWS JUST FIRED THEIR MOST CONTROVERSIAL ANCHOR #FoxNews #DonaldTrump #PresidentTrump #ShepSmith …
1964 : #DonaldTrump , For Now, Is Not Invited to Harry and Megha

3125 : Do you think Donald Trump did a good job addressing the shooting that took place in Parkland? #Parkland #Florida #POTUS #DonaldTrump
3126 : Dear Evangelicals,  #donaldtrump , apparently pays for sex, pays  #lovers  off and cheats on each of his wives. Why do you continue to support him? The  #Russian #prostitute  story doesn’t seem so farfetched, does it?   #trump #maga #trumprussia #christians #evangelical #civicdirect
3127 : #EmmaGonzalez  has more courage and heart than the entire  #GOP  combined!  #DonaldTrump  blamed the victims for this terrible  #FloridaShooting  tragedy. Now teenagers and teachers are on the frontlines as our government hides again. An uprising has begun! …
3128 : A four-star general who willfully does the dirty work of a five-deferment draft dodger should be stripped of his rank. Gen. Kelly’s a henchman for hate. #JohnKelly #GeneralMalfeasance #DonaldTrump #PretenderInChief
3129 : Has  #DonaldTrump  passed his security clearance? With all of his lies, r

4292 : Where is due process when it comes to  #DonaldTrump 's view on gun control?  #TMZ #BehindTheBar
4293 : Happy  #Womensday  to the women who give so much but don’t get enough back. Be Blessed, Be happy & Enjoy life #ImpeachTrump #Resist #impeachment #DonaldTrump
4294 : Which Twitter rules just got broken by Trump?       by  @imoyse  via  @DigitalLeadersA #socialmedia #socialselling #marketing #trump #donaldtrump #breakingtherules #bullying
4295 : I see the Resemblance.. only Difference Is  #chucky  Is A Evil Doll &  #DonaldTrump  Is A Evil Puppet  #TheResistance #Resistance #ImpeachTrump #NotMyPresident #Resist #Trump #Chucky #DonaldTrump #Memes #Funny #lol #Truth #EvilDoll #EvilPresident #DemForce #VoteThemOut2018 #BlueWave
4296 : So weird that we haven’t heard one peep from . @newtgingrich  regarding  #DonaldTrump  and his affair with hookers. This time there are possibly pics () and text. Yet the poster child for infidelity and religion has been silent, yet harped viciously on 

5458 : President Trump warns Congress about signing $1.3 trillion omnibus spending bill. #DonaldTrump #OmnibusSpendingBill …
5459 : Number 3 is most likely the number 1 reason.  #DonaldTrump  won't pay his attorneys
5460 : "What if we were just a regular father and daughter... with benefits?" #DonaldTrump #IvankaTrump #AntiIntellectualism #MuellerInvestigation #Obstruction #Impeach45 #CadetBoneSpurs #StableGenius #TrumpColluded #TheResistance #RussiaGate #Resist #Moron #Dotard #GOP #MAGA #AltRight
5461 : White House Probes Loans to Kushner’s Business - WSJ  #Trump #news #usa #twitter #cnn #Donaldtrump #MuellerTime  …
5462 : Interestingly enough the  @Forbes  article that debuted the  #HedyLamarr  story to “mainstream news” in the May 14, 1990 issue featured  #DonaldTrump  on the cover with the headline, “How much is Donald Trump really worth now?
5463 : I mean, Idk. But from all the  #headlines , all the 1st hand accounts from various people, all the scandals, you just get the feeling 

6291 : Hey Ontario you now have the only person I know who makes  #DonaldTrump  look like a genius  #DougFord   "Hydro One
6292 : Congrats to  @TyraSanchez  on being the  #DonaldTrump  of Drag.
6293 : So Mrs May, you want to join in with  #DonaldTrump  to bomb  #Syria  seriously risking escalation into a much wider, far more dangerous conflict, while denying the UK parliament a vote on it?  #Youmustbefuckingjoking #NotInMyNameTheresaMay
6294 : . @Telegraph  cartoon #DonaldTrump #TheresaMay #SyriaCrisis
6295 : #TheresaTheAppeaser #TheresaMay #DonaldTrump #WarMongers  as disgusting as all the others who suck up trumps arse...........   #SyriaUnderAttack #StopTheWar #Syria #WarOnIslam #WarOnMuslims #Crusaders #SyriaHoax #FalseFlag #ChemicalAttacks #Russia #ToriesOut #MayMustGo #VoteLabour
6296 : @realDonaldTrump  Attention  #DonaldTrump #SeniorsWillComeAfterYOU  if you cut OUR  #Medicare #Medicare4All #KeepMedicare #SeniorsForMedicare
6297 : Published February 2017 #RACISM #BIGOTRY #White

7458 : It's obstruction of Justice, not obstruction of a the investigation of a crime. Read a book !  #DonaldTrump #TrumpRussia #TheResistance
7459 : #BarackObama #CrookedHillary #BadIranDeal #ClintonFoundation #DeepStateOperatives #IranNuclearDeal #ObamaLegacy #LiberalismFindACure  #PeaceThroughStrength  Brought to US by @netanyahu #BenjaminNetanyahu @realDonaldTrump #DonaldTrump  #RNRTN  #MAGA_45 #MAGAWinning …
7460 : #DonaldTrump , the perpetual victim
7461 : US President  #DonaldTrump  suggests that his planned meeting with North Korean leader  #KimJongUn  be held in truce village at  border separating two Koreas.
7462 : Oh, the zombies are SO mean! It's enough to make even POTUS cry.  #indiegame #indiedev #donaldtrump #POTUS #funwithzombies #zombiegame #gaming #gamer #freetoplay #android #lowpoly #FREE
7463 : #BREAKING :  #ROBERTMUELLER  HAS QUESTIONS FOR  #DONALDTRUMP  ON  #JAMESCOMEY  FIRING    via  @YouTube
7464 : It's time for TRUMP to REALLY RUN through this dark ZOMBIE Zone!

8624 : AYOOOO, WHO DID THIS? Kids get scared of Donald Trump! #Meme #Vine #DonaldTrump #School #Kids #HipHop #Rap #CardiB #NickiMinaj #RemyMa #LilKim #Tupac #Biggie #Nas #DMX #NWA #KRS1 #BigDaddyKane #BDP #BTnH #Music #Talent #RoyalWedding #MeghanMarkle #JayZ #Beyonce #Illuminati
8625 : Lesley Stahl says she once asked  #DonaldTrump  why he  relentlessly attacks press #trump 's answer: “I do it to discredit you all and demean you all so that when you write negative stories about me, no one will believe you."
8626 : I fear the latter. And you should know at the very least it will take the will that my generation had to endure the Great Depression, fight fascism in the Second World War and demand a peace where all lived in dignity and prosperity. This is not going to be easy.    #DonaldTrump …
8627 : The meeting was bad for optics, it shouldn’t have happened. Bill Clinton was a private citizen with zero power.  #DonaldTrump ’s lap dogs are given access to documents that affect his case, 

9791 : #DonaldTrump  Appeared In A 2000  #PlayboySoftcorePorn … [[[  #RudyGiuliani "I’m sorry I don’t respect a  #PornStar " …  ]]]
9792 : #DonaldTrump  Ass muncher ! You orange monkey face !  Little dick What you gonna block me because I’m calling you names !! Honestly wish you would get your head out of your ass !! Pretty sad your wife cry’s for help on tv ! I thinkk you should step down and pack your bags
9793 : 
9794 : Today I graduated 8th grade and im proud of myslef and so are other people, but they spelled my right but wrong because its spelled differently thx  #DonaldTrump  btw its just a joke
9795 : We keep having to say this:You are not King  #DonaldTrump  &  #RobertMueller  doesn’t have any  #ConflictsOfInterest .This and other Mueller myths, debunked: …
9796 : Union says  #DonaldTrump  civil service orders violate federal labor law    ($)
9797 : Is this a grown man ‘speaking?!’ International trade is so much more complex than this, &  #DonaldTrump  is complaining that the 

In [17]:
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from string import digits
nltk.download('stopwords')


/home/miguel/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


[nltk_data] Downloading package stopwords to /home/miguel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:

tokenizer = RegexpTokenizer(r'\w+')
documents = [ tokenizer.tokenize(t.lower()) for t in tweet_no_html ]

In [19]:
#def clean_tweet(tweet):
    
#def tokenize(tweet):
    

In [20]:
#tokenized_and_clean_tweets = [tokenize(clean_tweet(tweet)) for tweet in tweet_no_html]

In [21]:
documents

[['dear', 'donaldtrump'],
 ['america', 'demands', 'trumpshowusyourpenis', 'donaldtrump', 'maga'],
 ['enough', 'donaldtrump'],
 ['donaldtrump',
  'stop',
  'blaming',
  'pak',
  'and',
  'pak',
  'should',
  'stopamericansupplies'],
 ['pakistan',
  'will',
  'not',
  'do',
  'more',
  'nomoretrump',
  'donaldtrump',
  'stopamericansupplies'],
 ['donaldtrump',
  'donaldtrumpspak',
  'short',
  'term',
  'memory',
  'loss',
  'syndrome',
  'or',
  'bipolar',
  'disorder'],
 ['pakistan',
  'is',
  'not',
  'fighting',
  'for',
  'money',
  'dg',
  'ispr',
  'nomoretrump',
  'stopamericansupplies',
  'donaldtrump'],
 ['dear',
  'donaldtrump',
  'you',
  'know',
  'fairly',
  'after',
  'march',
  '2018',
  'you',
  'will',
  'never',
  'be',
  'able',
  'to',
  'threatened',
  'state',
  'of',
  'pakistan',
  'ever',
  'again',
  'so',
  'now',
  'nomoretrump'],
 ['so',
  'the',
  'president',
  'of',
  'america',
  'doesn',
  't',
  'know',
  'that',
  'the',
  'exeggerated',
  'figure',
 

In [22]:
unigrams = [ w for doc in documents for w in doc if len(w)==1]
bigrams  = [ w for doc in documents for w in doc if len(w)==2]

In [23]:
trump_stop_words = ['trump','donald','donaldtrump','realdonaldtrump']

In [24]:
stoplist  = set(nltk.corpus.stopwords.words("english") + 
                 unigrams + bigrams + trump_stop_words)

In [25]:
documents = [[token for token in doc if token not in stoplist and not token.isdigit()] for doc in documents]

In [26]:
#documents

In [27]:
for doc in documents:
    doc.sort()

dictionary = corpora.Dictionary(documents)
dictionary.compactify()
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [28]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:10]]

[[('dear', 1)],
 [('america', 1), ('demands', 1), ('maga', 1), ('trumpshowusyourpenis', 1)],
 [('enough', 1)],
 [('blaming', 1), ('pak', 2), ('stop', 1), ('stopamericansupplies', 1)],
 [('stopamericansupplies', 1), ('nomoretrump', 1), ('pakistan', 1)],
 [('bipolar', 1),
  ('disorder', 1),
  ('donaldtrumpspak', 1),
  ('loss', 1),
  ('memory', 1),
  ('short', 1),
  ('syndrome', 1),
  ('term', 1)],
 [('stopamericansupplies', 1),
  ('nomoretrump', 1),
  ('pakistan', 1),
  ('fighting', 1),
  ('ispr', 1),
  ('money', 1)],
 [('dear', 1),
  ('nomoretrump', 1),
  ('pakistan', 1),
  ('able', 1),
  ('ever', 1),
  ('fairly', 1),
  ('know', 1),
  ('march', 1),
  ('never', 1),
  ('state', 1),
  ('threatened', 1)],
 [('america', 1),
  ('know', 1),
  ('aid', 1),
  ('billion', 1),
  ('coalition', 1),
  ('exeggerated', 1),
  ('figure', 1),
  ('fund', 1),
  ('president', 1),
  ('support', 1)],
 [('disappeare', 1), ('problem', 1)]]

In [29]:
'''lda_model =  models.ldamodel.LdaModel(corpus, num_topics=30)
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus,dictionary)'''

'lda_model =  models.ldamodel.LdaModel(corpus, num_topics=30)\npyLDAvis.enable_notebook()\npyLDAvis.gensim.prepare(lda_model, corpus,dictionary)'

In [30]:
lda_model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary)

In [31]:
topic_words = lda_model.show_topics(formatted=False)
print(topic_words[1])

(1, [('president', 0.010419962), ('kimjongun', 0.007426276), ('usa', 0.0068267905), ('trumpkimsummit', 0.006468086), ('canada', 0.006068152), ('north', 0.004786145), ('korea', 0.0043517095), ('people', 0.0037318051), ('justintrudeau', 0.0035605803), ('man', 0.0034741575)])


In [32]:
x=lda_model.show_topics(num_topics=12, num_words=5,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words 
for topic,words in topics_words:
    print(" ".join(words))

0::['singapore', 'president', 'maga', 'potus', 'foxnews']
1::['president', 'kimjongun', 'usa', 'trumpkimsummit', 'canada']
2::['kimjongun', 'trumpkimsummit', 'singaporesummit', 'president', 'summit']

singapore president maga potus foxnews
president kimjongun usa trumpkimsummit canada
kimjongun trumpkimsummit singaporesummit president summit


In [33]:
topic_words_per_document.append(topic_words)

In [34]:
topic_words_per_document

[[(0,
   [('singapore', 0.012361183),
    ('president', 0.009280885),
    ('maga', 0.0071136784),
    ('potus', 0.00530841),
    ('foxnews', 0.0042490005),
    ('trumpkimsummit', 0.003925247),
    ('america', 0.003914129),
    ('body', 0.0037126432),
    ('cnn', 0.0036449907),
    ('analysis', 0.0035089129)]),
  (1,
   [('president', 0.010419962),
    ('kimjongun', 0.007426276),
    ('usa', 0.0068267905),
    ('trumpkimsummit', 0.006468086),
    ('canada', 0.006068152),
    ('north', 0.004786145),
    ('korea', 0.0043517095),
    ('people', 0.0037318051),
    ('justintrudeau', 0.0035605803),
    ('man', 0.0034741575)]),
  (2,
   [('kimjongun', 0.016140422),
    ('trumpkimsummit', 0.008114253),
    ('singaporesummit', 0.007860324),
    ('president', 0.0070596416),
    ('summit', 0.006467416),
    ('northkorea', 0.005222512),
    ('potus', 0.005012077),
    ('world', 0.0048545618),
    ('america', 0.0048336107),
    ('one', 0.004543916)])]]